# Two chained reservoirs

Two disconnected electrical loads are fed from two reservoirs linked by a river; the first reservoir has inflow from rain onto a water basin.

Note that the two reservoirs are tightly coupled, meaning there is no time delay between the first one emptying and the second one filling, as there would be if there were a long stretch of river between the reservoirs. The reservoirs are essentially assumed to be close to each other. A time delay would require a "Link" element between different snapshots, which is not yet supported by PyPSA (but could be enabled by passing `network.optimize()` an extra_functionality function).

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import pypsa

In [ ]:
network = pypsa.Network()
network.set_snapshots(pd.date_range("2016-01-01 00:00", "2016-01-01 03:00", freq="H"))

Add assets to the network.

In [ ]:
network.add("Carrier", "reservoir")
network.add("Carrier", "rain")

network.add("Bus", "0", carrier="AC")
network.add("Bus", "1", carrier="AC")

network.add("Bus", "0 reservoir", carrier="reservoir")
network.add("Bus", "1 reservoir", carrier="reservoir")


network.add(
    "Generator",
    "rain",
    bus="0 reservoir",
    carrier="rain",
    p_nom=1000,
    p_max_pu=[0.0, 0.2, 0.7, 0.4],
)


network.add("Load", "0 load", bus="0", p_set=20.0)

network.add("Load", "1 load", bus="1", p_set=30.0)

The efficiency of a river is the relation between the gravitational potential energy of 1 m^3 of water in reservoir 0 relative to its turbine versus the potential energy of 1 m^3 of water in reservoir 1 relative to its turbine

In [ ]:
network.add(
    "Link",
    "spillage",
    bus0="0 reservoir",
    bus1="1 reservoir",
    efficiency=0.5,
    p_nom_extendable=True,
)


# water from turbine also goes into next reservoir
network.add(
    "Link",
    "0 turbine",
    bus0="0 reservoir",
    bus1="0",
    bus2="1 reservoir",
    efficiency=0.9,
    efficiency2=0.5,
    capital_cost=1000,
    p_nom_extendable=True,
)

network.add(
    "Link",
    "1 turbine",
    bus0="1 reservoir",
    bus1="1",
    efficiency=0.9,
    capital_cost=1000,
    p_nom_extendable=True,
)


network.add(
    "Store", "0 reservoir", bus="0 reservoir", e_cyclic=True, e_nom_extendable=True
)

network.add(
    "Store", "1 reservoir", bus="1 reservoir", e_cyclic=True, e_nom_extendable=True
)

In [ ]:
network.optimize(network.snapshots)
print("Objective:", network.objective)

In [ ]:
network.generators_t.p.plot.area(figsize=(9, 4))
plt.tight_layout()

Now, let's have  look at the different outputs of the links.

In [ ]:
network.links_t.p0.plot(figsize=(9, 4), lw=3)
plt.tight_layout()

In [ ]:
network.links_t.p1.plot(figsize=(9, 4), lw=3)
plt.tight_layout()

In [ ]:
network.links_t.p2.plot(figsize=(9, 4), lw=3)
plt.tight_layout()

What are the energy outputs and energy levels at the reservoirs?

In [ ]:
pd.DataFrame({attr: network.stores_t[attr]["0 reservoir"] for attr in ["p", "e"]})

In [ ]:
pd.DataFrame({attr: network.stores_t[attr]["1 reservoir"] for attr in ["p", "e"]})